본 서적의 깃허브 및 블로그 주소 </br>
https://bit.ly/transformer-git </br>
https://bit.ly/transformer-home


In [4]:
# Colab에는 트랜스포머가 설치되어 있지 않으므로 이 코드를 사용해 수동 설치
!pip install transformers

In [5]:
# 트랜스포머에서는 pipeline() 함수를 호출하면서 관심작업 이름을 전달해 파이프라인 객체를 만든다.
from transformers import pipeline

classifier = pipeline("text-classification")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


처음 위 코드를 실행하면 파이프라인이 자동으로 허깅페이스 허브에서 모델 가중치를 다운로드 한다.  
허깅 페이스 허브 주소  
https://oreil.ly/zLK11

</br>

파라미터인 text-classification은 감성 분석을 위한 모델을 사용하지만  
다중분류와 다중 레이블 분류도 지원한다.  
  
이제 파이프라인이 있으니 예측을 만들어보자

In [7]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

우선 위와 같은 텍스트를 준비했다.  
이것은 온라인 주문에 대한 가짜 피드백으로 법률 계약, 제품 설명 등의 내용이 들어가 있다.  
이것은 과연 긍정적인 피드백인가 부정적인 피드백인가?  
</br>
한 번 분류해보자

In [8]:
import pandas as pd

outputs = classifier(text)
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.901546


각 예측은 하나의 파이썬 딕셔너리 이므로 판다스를 사용한다.  
DataFrame 메소드를 통해 결과를 출력해본 결과 부정적이라는 결과가 나온다.  

</br>

자, 이걸 분석해보자